In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics as sm
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression,LinearRegression,Ridge,Lasso

In [3]:
df=pd.read_csv("credit_train.csv")

NameError: name 'credit_train' is not defined

In [ ]:
df.head(6)

In [ ]:
df.shape

In [ ]:
df.dtype()

In [ ]:
df.isnull().sum()

__Imputing na values__

In [ ]:
df.drop(['Months since last delinquent','Loan ID', 'Customer ID'],axis=1,inplace=True)

In [ ]:
df['Credit Score'].fillna(0,inplace=True)

In [ ]:
df["Current Loan Amount"].fillna(value=df["Current Loan Amount"].median(),inplace=True)
#df["Credit Score"].fillna(value=df["Credit Score"].median(),inplace=True)

In [ ]:
df["Annual Income"].fillna(value=df["Annual Income"].median(),inplace=True)

In [ ]:
df.columns

__dropping NAN values__

### Dropping NA values

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
data=df.copy()

In [ ]:
data.head()

In [ ]:
import seaborn as sns
def heatMap(df):
    #Create Correlation df
    corr = df.corr()
    #Plot figsize
    fig, ax = plt.subplots(figsize=(10, 7))
    #Generate Heat Map, allow annotations and place floats in map
    sns.heatmap(corr, annot=True, fmt=".2f")
    #Apply xticks
    plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
    plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
    plt.show()

In [ ]:
heatMap(data)

In [ ]:
#

### Distribution of Loan Status

In [ ]:
ls_vc=data['Loan Status'].value_counts()
print(ls_vc)
ls_vc.plot(kind='bar',grid=True,figsize=(10,7))

In [ ]:
data['Annual Income'].plot(figsize=(10,7))

In [ ]:
print(data['Annual Income'].min())
print(data['Annual Income'].max())

In [ ]:
data['Annual Income Range']=pd.cut(data['Annual Income'],[50000,500000,1000000,2000000,200000000],
                                   labels=['Lower Middle Class','Middle Class','Upper Middle Class','High Class'],right=True)

In [ ]:
#data.groupby(['Purpose']).plot(kind='bar',grid=True)

In [ ]:
data['Annual Income Range'].value_counts().plot(kind='bar',grid=True)

In [ ]:
data.groupby(['Loan Status']).size().plot(kind='bar')

In [ ]:
df.groupby(['Term','Loan Status']).size().unstack().plot(kind='bar',grid=True)

In [ ]:
data.columns

In [ ]:
data.pivot_table(index='Loan Status',aggfunc='mean',dropna=True)

__Creating Risky Column__

In [ ]:
def mapping(data):
    if data>=720:
        return 'Low Risky'
    else:
        return 'High Risky'

In [ ]:
data['Loan Risky']=data['Credit Score'].apply(mapping)

In [ ]:
data['Number of Open Accounts'].unique()

In [ ]:
data.groupby(['Number of Open Accounts','Loan Risky']).size().unstack().plot(kind='bar',grid=True,figsize=(15,7))

In [ ]:
data.groupby(['Loan Risky']).size().plot(kind='bar',grid=True)

__Want to Predict Credit Score__

In [ ]:
#So i selected only those rows where credit score is 0 as my test data inorder to predict the credit score
test_data=df[df['Credit Score']==0]

In [ ]:
test_data.shape

In [ ]:
#and remaining all as train data
train_data=df[df['Credit Score']!=0]

In [ ]:
train_data.shape

In [ ]:
train_data.drop('Loan Status',axis=1,inplace=True)

In [ ]:
train_data.columns

In [ ]:
# Splitting data manually as x_train and y_train inorder to predict credit score using linear regression

In [ ]:
credit_train_y=train_data['Credit Score']
credit_train_x=train_data[['Current Loan Amount', 'Term',
        'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Number of Open Accounts', 'Number of Credit Problems',
       'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies',
       'Tax Liens']]

In [ ]:
credit_cat=credit_train_x.select_dtypes(include='object')
credit_num=credit_train_x.select_dtypes(include='number')

In [ ]:
credit_cat=pd.get_dummies(credit_cat)
credit_train_x=pd.concat([credit_cat,credit_num],axis=1)

In [ ]:
credit_train_x.head(10)

In [ ]:
test_data.drop('Loan Status',axis=1,inplace=True)

In [ ]:
test_data.columns

In [ ]:
credit_test_y=test_data['Credit Score']
credit_test_x=test_data[['Current Loan Amount', 'Term',
        'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Number of Open Accounts', 'Number of Credit Problems',
       'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies',
       'Tax Liens']]

In [ ]:
credit_test_cat=credit_test_x.select_dtypes(include='object')
credit_test_num=credit_test_x.select_dtypes(include='number')

In [ ]:
credit_test_cat=pd.get_dummies(credit_test_cat)
credit_test_x=pd.concat([credit_test_cat,credit_test_num],axis=1)

In [ ]:
# building a linear regression model

In [ ]:
lin_model=LinearRegression().fit(credit_train_x,credit_train_y)
predicted_Credit_scores=lin_model.predict(credit_test_x)

In [ ]:
#predicted credit scores in place of nan values
predicted_Credit_scores[:20]

In [ ]:
# creating new column as credit score in test data frame

In [ ]:
credit_test_x['Credit Score']=predicted_Credit_scores

In [ ]:
print("Mean Squared Error:",mean_squared_error(credit_test_y,predicted_Credit_scores))
print("R2 value:",r2_score(credit_test_y,predicted_Credit_scores))

In [ ]:
# creating new column as credit score in train data frame

In [ ]:
credit_train_x['Credit Score']=credit_train_y

In [ ]:
# concatinating train and teat data horizontally as a total data

In [ ]:
total_data=pd.concat([credit_train_x,credit_test_x],axis=0)

In [ ]:
total_data.head(5)

In [ ]:
df['Loan Status'].value_counts()

In [ ]:
total_data['Loan Status']=df['Loan Status'].replace(to_replace=["Fully Paid","Charged Off"],value=["1","0"])

In [ ]:
total_data.reset_index(drop=True,inplace=True)

In [ ]:
total_data.head(5)

__Decision Tree to predict Loan Status__

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from scikitplot.metrics import plot_roc_curve

In [ ]:
target= total_data['Loan Status']
ind_vars= total_data.loc[:,total_data.columns!='Loan Status']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(ind_vars,target,test_size=0.2,random_state=35)

In [ ]:
lg_model=LogisticRegression().fit(x_train,y_train)
lg_predicted_values=lg_model.predict(x_test)

In [ ]:
print(sm.accuracy_score(y_test,lg_predicted_values)*100)

In [ ]:
print(sm.confusion_matrix(y_test,lg_predicted_values))

In [ ]:
print(sm.classification_report(y_test,lg_predicted_values))

In [ ]:
decisionTree_model=DecisionTreeClassifier(criterion='gini').fit(x_train,y_train)
train_predictions=decisionTree_model.predict(x_train)
test_predictions=decisionTree_model.predict(x_test)

In [ ]:
print("Decision Tree Train Accuracy:",sm.accuracy_score(y_train,train_predictions))
print("Decision Tree Test Accuracy:",sm.accuracy_score(y_test,test_predictions))

In [ ]:
print(sm.classification_report(y_test,test_predictions))

In [ ]:
y_pred_probs = decisionTree_model.predict_proba(x_test)
plot_roc_curve(y_test, y_pred_probs, curves=['each_class'], figsize=(10,7))
plt.show()

__Random Forest Classifier__

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  cross_val_score,GridSearchCV

In [ ]:
RF_clf = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1, class_weight="balanced",oob_score=True)
RF_clf.fit(x_train,y_train)

In [ ]:
rf_train_predictions=RF_clf.predict(x_train)
print("Random Forest Train Accuracy:",sm.accuracy_score(y_train,rf_train_predictions)*100)

rf_predictions=RF_clf.predict(x_test)
print("Random Forest Test Accuracy:",sm.accuracy_score(y_test,rf_predictions)*100)

__Top 10 Important Features__

In [ ]:
imp_features=RF_clf.feature_importances_
cols=x_train.columns
important_features=[]
for feat_names in zip(cols,imp_features):
    important_features.append(feat_names)

top_10_features=sorted(important_features, key=lambda x: x[1],reverse=True)[:10]
plt.bar(*zip(*top_10_features))
plt.xticks(rotation=90)
plt.show()

__Fine Tuning Using Grid Search CV__

In [ ]:
params = {'max_depth': range(1,10,2),
          'max_features': ['auto', 'sqrt','log2'],
          'n_estimators': [200, 400, 600, 800, 1000, 1200],
          'min_samples_leaf': [1, 2, 4],
          'min_samples_split': [2, 5, 10]
         }

Grid_randomF = GridSearchCV(RandomForestClassifier(random_state=15), params)

Grid_randomF.fit(x_train, y_train)

print(Grid_randomF.best_params_)

In [ ]:
clf = RandomForestClassifier(random_state=0, n_jobs=-1, class_weight="balanced",oob_score=True,**Grid_randomF.best_params_)
clf.fit(x_train,y_train)

In [ ]:
clf_score = cross_val_score(clf, x_train, y_train, cv=10)
print(clf_score)
clf_score.mean()